#### 2353. Design a Food Rating System

* https://leetcode.com/problems/design-a-food-rating-system/

In [11]:
# https://www.youtube.com/watch?v=Ikp8SgbgbEo

from sortedcontainers import SortedSet
from collections import defaultdict

class FoodRatings:

    def __init__(self, foods, cuisines, ratings):
        self.cuisines_ratings_foods = defaultdict(SortedSet)
        self.food_cuisines = {}
        self.food_ratings = {}

        for i in range(len(foods)):
            self.cuisines_ratings_foods[cuisines[i]].add((-ratings[i], foods[i])) # top rating at the first element
            self.food_cuisines[foods[i]] = cuisines[i]
            self.food_ratings[foods[i]] = ratings[i]

    def changeRating(self, food, newRating):
        c = self.food_cuisines[food]
        r = self.food_ratings[food]
        self.cuisines_ratings_foods[c].remove((-r, food))
        self.cuisines_ratings_foods[c].add((-newRating, food))
        self.food_ratings[food] = newRating

    def highestRated(self, cuisine):
        return self.cuisines_ratings_foods[cuisine][0][1]


foodRatings = FoodRatings(["kimchi", "miso", "sushi", "moussaka", "ramen", "bulgogi"], ["korean", "japanese", "japanese", "greek", "japanese", "korean"], [9, 12, 8, 15, 14, 7])
foodRatings.highestRated("korean")
foodRatings.highestRated("japanese")
foodRatings.changeRating("sushi", 16)
foodRatings.highestRated("japanese")
foodRatings.changeRating("ramen", 16)
foodRatings.highestRated("japanese")

'ramen'

In [12]:
### heapq solution
from collections import defaultdict
import heapq
class FoodRatings:

    def __init__(self, foods, cuisines, ratings):
        self.cuisines_ratings_foods_heap = defaultdict(list)
        self.food_cuisines = {}
        self.food_ratings = {}

        for food, cuisine, rating in zip(foods, cuisines, ratings):
            self.food_cuisines[food] = cuisine
            self.food_ratings[food] = rating
            heapq.heappush(self.cuisines_ratings_foods_heap[cuisine], (-rating, food)) # separate heap for each cuisine

    def changeRating(self, food, newRating):
        self.food_ratings[food] = newRating
        cuisine = self.food_cuisines[food]
        heapq.heappush(self.cuisines_ratings_foods_heap[cuisine], (-newRating, food))

    def highestRated(self, cuisine):
        heap = self.cuisines_ratings_foods_heap[cuisine]
        while heap:
            rating, food = heap[0]
            if -rating == self.food_ratings[food]:
                return food
            else:
                heapq.heappop(heap)


foodRatings = FoodRatings(["kimchi", "miso", "sushi", "moussaka", "ramen", "bulgogi"], ["korean", "japanese", "japanese", "greek", "japanese", "korean"], [9, 12, 8, 15, 14, 7])
foodRatings.highestRated("korean")
foodRatings.highestRated("japanese")
foodRatings.changeRating("sushi", 16)
foodRatings.highestRated("japanese")
foodRatings.changeRating("ramen", 16)
foodRatings.highestRated("japanese")

'ramen'

 Key Optimizations:
Max Heap simulation with (-rating, food) to keep top-rated foods on top.

Lazy deletion: When updating ratings, we just push the new value; old values are ignored during highestRated.

Lexicographical tie-breaking handled automatically due to ( -rating, food ) tuple ordering.


🔁 Why Do We Need a while Loop in highestRated?
Yes, heapq ensures the top of the heap contains the food with the highest rating (since we use (-rating, food) for max-heap behavior).
BUT, when we do a changeRating, we don’t remove the old value from the heap (heapq has no O(log n) remove), so we push a new entry and leave the old one behind — this is called lazy deletion.

⚠️ Problem Scenario Without while Loop
Here’s what can go wrong without the loop:

Initial:

python
Copy
Edit
sushi = 10  => heap = [(-10, 'sushi')]
You call changeRating("sushi", 20)
→ New entry is pushed:

python
Copy
Edit
heap = [(-20, 'sushi'), (-10, 'sushi')]
Top of heap is now (-20, 'sushi') ✅ correct!

But suppose you change it again:

python
Copy
Edit
changeRating("sushi", 15)
heap = [(-20, 'sushi'), (-10, 'sushi'), (-15, 'sushi')]
Top of the heap is still (-20, 'sushi'), but now this entry is outdated — current rating is 15.

So when you call highestRated('japanese'), you must loop until the top of the heap has a rating that matches food_to_rating[food].

✅ That’s Why We Do:
python
Copy
Edit
while heap:
    rating, food = heap[0]
    if -rating == self.food_to_rating[food]:
        return food
    else:
        heapq.heappop(heap)
We pop outdated entries until we find the most up-to-date (rating, food) pair.